In [3]:
import numpy as np
import pandas as pd
import pickle

# 1. Data
- Load data from pickle file
- split the data into independent and dependent
- split to train and test set

In [4]:
data = pickle.load(open('data_face_features_emotion.pickle',mode='rb'))

In [5]:
X = np.array(data['data']) 
y = np.array(data['label'])

In [6]:
X.shape , y.shape

((1413, 1, 128), (1413,))

In [7]:
X = X.reshape(-1,128)
X.shape

(1413, 128)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)

In [10]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1130, 128), (283, 128), (1130,), (283,))

# 2. Train Machine Learning

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

### Logistic Regression

In [12]:
model_logistic = LogisticRegression()
model_logistic.fit(x_train,y_train) 

LogisticRegression()

In [13]:
def get_report(model, x_train,y_train,x_test,y_test):
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)

    # accuracy score
    acc_train = accuracy_score(y_train,y_pred_train)
    acc_test = accuracy_score(y_test,y_pred_test)

    # f1 score
    f1_score_train = f1_score(y_train,y_pred_train,average='macro')
    f1_score_test = f1_score(y_test,y_pred_test,average='macro')


    print('Accuracy Train = %0.2f'%acc_train)
    print('Accuracy Test = %0.2f'%acc_test)
    print('F1 Score Train = %0.2f'%f1_score_train)
    print('F1 Score Test = %0.2f'%f1_score_test)

In [14]:
get_report(model_logistic,x_train,y_train,x_test,y_test)

Accuracy Train = 0.29
Accuracy Test = 0.27
F1 Score Train = 0.22
F1 Score Test = 0.18


### Support Vector Machines

In [15]:
model_svc = SVC(probability=True)
model_svc.fit(x_train,y_train)

SVC(probability=True)

In [16]:
get_report(model_svc,x_train,y_train,x_test,y_test)

Accuracy Train = 0.40
Accuracy Test = 0.27
F1 Score Train = 0.31
F1 Score Test = 0.18


### Random Forest

In [17]:
model_rf = RandomForestClassifier(n_estimators=10,)
model_rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=10)

In [18]:
get_report(model_rf,x_train,y_train,x_test,y_test)

Accuracy Train = 0.99
Accuracy Test = 0.45
F1 Score Train = 0.99
F1 Score Test = 0.48


## Voting Classifier

In [19]:
model_voting = VotingClassifier(estimators=[
    ('logistic',LogisticRegression()),
    ('svm',SVC(probability=True)),
    ('rf',RandomForestClassifier())
], voting='soft',weights=[2,3,1])

In [20]:
model_voting.fit(x_train,y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [21]:
get_report(model_voting,x_train,y_train,x_test,y_test)

Accuracy Train = 0.88
Accuracy Test = 0.44
F1 Score Train = 0.89
F1 Score Test = 0.41


# 3. Parameter Tuning

In [22]:
from sklearn.model_selection import GridSearchCV

In [23]:
model_grid = GridSearchCV(model_voting,
                         param_grid={
                             'svm__C':[3,5,7,10],
                             'svm__gamma':[0.1,0.3,0.5],
                             'rf__n_estimators':[5,10,20],
                             'rf__max_depth':[3,5,7],
                             'voting':['soft']
                         },scoring='accuracy',cv=3,n_jobs=1,verbose=2)

In [24]:
model_grid.fit(x_train,y_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.7s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier())],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft']},
             scoring='accuracy', verbose=2)

In [25]:
model_best_estimator = model_grid.best_estimator_

In [26]:
model_grid.best_score_

0.29469778204187597

# 4. Save Model

In [28]:
pickle.dump(model_best_estimator,open('./models/machinelearning_face_emotion.pkl',mode='wb'))